# OpenAI Responses API

## What is the OpenAI Responses API?

The Responses API is a new API released in March 2025. It is a combination of the traditional 
Chat Completions API and the Assistants API, providing support for:

- **Traditional Chat Completions:** Facilitates seamless conversational AI experiences.
- **Web Search:** Enables real-time information retrieval from the internet.
- **File Search:** Allows searching within files for relevant data.

Accordingly, the Assistants API will be retired in 2026. 

> **For new users, OpenAI recommends using the Responses API instead of the Chat Completions API to leverage its expanded capabilities.**

For a comprehensive comparison between the Responses API and the Chat Completions API, refer to the official OpenAI documentation: 
[Responses vs. Chat Completions](https://platform.openai.com/docs/guides/responses-vs-chat-completions).

## Summary of This Notebook
This notebook provides a hands-on guide for using the **OpenAI Responses API** to analyze tweets. 
It covers essential techniques such as:

- **Creating a vector store** and uploading tweets for semantic search.
- **Using file search** to analyze private datasets.
- **Performing a web search** to retrieve the latest public information.
- **Utilizing stateful responses** to maintain conversation context.
- **Combining file and web search** to enhance retrieval-augmented generation (RAG) applications.

By the end of this notebook, users will be able to integrate OpenAI's Responses API for efficient data retrieval and analysis of structured and unstructured data.

## Install Required Libraries
To use the OpenAI Responses API, we need to install the following libraries:

- **`openai`**: Provides access to OpenAI's APIs, including the Responses API

In [1]:
pip install openai -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.3.3 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


## Import Required Libraries

In [2]:
from IPython.display import Markdown, display
import boto3
from botocore.exceptions import ClientError
import json
import io

## Retrieve Secrets from AWS Secrets Manager

In [3]:
def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

## Initialize OpenAI Client

In [4]:
from openai import OpenAI
openai_api_key  = get_secret('openai')['api_key']

client = OpenAI(api_key=openai_api_key)

## File Search API

### Introduction to File Search
File search API enables efficient retrieval of relevant information 
from uploaded files by leveraging vector-based indexing. This feature is particularly useful 
for searching large datasets, extracting insights, and improving retrieval-augmented generation (RAG) applications.

Unlike traditional keyword-based searches, the Responses API uses embeddings 
to identify semantically relevant content, making it ideal for analyzing structured 
and unstructured text data (OpenAI, 2025).

For more details, visit the official OpenAI documentation: 
[File Search in Responses API](https://platform.openai.com/docs/guides/tools-file-search).

### Create a Vector Store

In [5]:
vector_store = client.vector_stores.create(
    name="my_vector_store"
)
vector_store_id = vector_store.id
print(vector_store_id)

vs_69150cb6ea88819184e62261b796f0cf


### Upload Files

In [6]:
with open('tweet_text.json', 'rb') as f:
    file = client.files.create(
        file=f,            # file-like object
        purpose="assistants"
    )

file_id = file.id
print(file_id)

file-BGJrFFYhQA49jSuUmueksS


### Attach File to Vector Store

In [7]:
attach_status =client.vector_stores.files.create(
    vector_store_id=vector_store_id,
    file_id=file_id
            )

print(attach_status.id)

file-BGJrFFYhQA49jSuUmueksS


### Query the Vector Store

In [8]:
query = "the latest development in generativeAI"

In [9]:
search_results = client.vector_stores.search(
    vector_store_id=vector_store_id,
    query=query
)

for result in search_results.data[:5]:
    print(result.content[0].text[:100] + '\n Relevant score: ' + str(result.score))

## OpenAI Response API

### Simple Response

In [10]:
simple_response = client.responses.create(
  model="gpt-4o",
  input=[
      {
          "role": "user",
          "content": query
      }
  ]
)

In [11]:
display(Markdown(simple_response.output_text))

As of the latest updates, generative AI is advancing rapidly with several key developments:

1. **Scaling of Models**: Transformers and language models like OpenAI's GPT series and Google's Bard have scaled massively, improving in complexity and performance.

2. **Multimodal Models**: AI systems like OpenAI's DALL-E, Google's Imagen, and Meta's Make-A-Scene are integrating text, images, and more, enabling the creation of high-quality visual content from textual descriptions.

3. **Improved Text Generation**: Models are becoming more adept at generating coherent and context-aware text, aiding in applications from content creation to conversational agents.

4. **Real-time Collaboration**: Generative AI is being used in real-time creative applications, allowing users to collaborate with AI to create music, art, and writing.

5. **AI Ethics and Safety**: There is a growing focus on developing ethical guidelines, reducing biases in AI output, and ensuring models are safe for deployment.

6. **Customization and Personalization**: Models are being fine-tuned for specific industries and applications, providing more relevant outputs tailored to user needs.

7. **Efficient Training Techniques**: New techniques are emerging to train models more efficiently, reducing the computational resources required and speeding up the development cycle.

These developments are pushing generative AI towards broader applicability and integration across various fields.

### File Search Response

In [12]:

file_search_response = client.responses.create(
    input= query,
    model="gpt-4o",
    temperature = 0,
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
    }]
)

In [13]:
display(Markdown(file_search_response.output_text))


The latest developments in generative AI include several key advancements:

1. **Agentic Workflows**: Amazon Web Services is exploring the future of AI with a focus on agentic workflows, which are being showcased alongside startups like NeuralSeek and Tarpit AI.

2. **AI in Supply Chain**: Atos has developed an AI-powered Supply Chain Disruption Analysis using generative AI, SAP BTP, and AWS Bedrock to assess risk and boost resilience.

3. **Cinematic Content Creation**: OpenAI's Sora2 is enabling the creation of cinematic videos from prompts, incorporating audio, physics, and cameos, which is a game-changer for creators.

4. **Enterprise Applications**: IBM's Watsonx is bringing generative AI to enterprises, allowing teams to build custom large language models for enhanced customer engagement and streamlined processes.

5. **Healthcare Transformation**: Generative AI is being used to support doctors with data interpretation and preliminary analysis, highlighting its potential in transforming healthcare.

These developments illustrate the broadening scope and impact of generative AI across various industries.

## Web Search API

### Introduction to Web Search
The OpenAI Web Search tool allows models to retrieve real-time information from the internet. 
This capability is particularly useful for obtaining up-to-date data, fact-checking, and expanding knowledge 
without relying solely on pre-trained information. 

By leveraging OpenAI's web search functionality, the Responses API can fetch external data 
and provide accurate, relevant results in real time (OpenAI, 2025). 
This feature enhances applications that require the latest insights, such as news aggregation, research, 
or dynamic content generation.

For more details, visit the official OpenAI documentation: 
[Web Search in Responses API](https://platform.openai.com/docs/guides/tools-web-search).

### Perform Web Search

In [14]:
web_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= query,
    tools=[
        {
            "type": "web_search"
        }
    ]
)

In [15]:
display(Markdown(web_search_response.output_text))

Here’s an in-depth update on the **latest developments in generative AI** as of **mid-November 2025**. This overview is structured into major domains: model releases, enterprise integrations, emerging trends, and foundational research.

---

###  Cutting-Edge Model Releases

**1. OpenAI’s GPT-5**  
On **August 7, 2025**, OpenAI launched **GPT-5**, a powerful multimodal foundation model accessible via ChatGPT, Microsoft Copilot, and the OpenAI API. GPT-5 merges advanced reasoning with multimodal capabilities—representing a significant leap over GPT‑4 and its variants ([en.wikipedia.org](https://en.wikipedia.org/wiki/GPT-5?utm_source=openai)).

**2. Google’s Gemini & “Nano Banana”**  
In **August 2025**, Google DeepMind released **Gemini 2.5 Flash Image**, widely known as **“Nano Banana”**—a text-to-image generation tool integrated into the Gemini app and Vertex AI. Renowned for its photorealistic “3D figurine” imagery, it enables dynamic editing and context-aware modifications via natural language, and features SynthID watermarking for content traceability. The release contributed to more than 200 million image edits and over 10 million first-time Gemini app users in the weeks following launch ([en.wikipedia.org](https://en.wikipedia.org/wiki/Nano_Banana?utm_source=openai)).

**3. Meta’s Llama 4 Series**  
Earlier in **April 2025**, Meta released the **Llama 4 Scout** and **Llama 4 Maverick**, part of a broader multimodal AI framework capable of processing text, images, audio, and video. An even more powerful model, **Llama 4 Behemoth**, was previewed, affirming Meta’s aggressive AI infrastructure expansion, with plans of up to $65 billion in investments to date ([reuters.com](https://www.reuters.com/technology/meta-releases-new-ai-model-llama-4-2025-04-05/?utm_source=openai)).

**4. OpenAI’s Open-Weight Releases: gpt-oss-20b & 120b**  
OpenAI opened new paths toward accessibility by releasing **gpt-oss-20b** and **gpt-oss-120b**, the first open-weight models since GPT-2. These support chain-of-thought reasoning, can run locally on consumer hardware (e.g., PCs with 16GB RAM or RTX GPUs), and are available across platforms like Hugging Face, Azure, and AWS ([windowscentral.com](https://www.windowscentral.com/artificial-intelligence/openai-chatgpt/openai-launches-two-gpt-models-theyre-not-gpt-5-but-they-run-locally-on-snapdragon-pcs-and-nvidia-rtx-gpus?utm_source=openai)).

**5. Other Notable Models**  
- February 2025 saw releases such as OpenAI’s **GPT‑4.5**, xAI’s **Grok 3**/Mini, Anthropic’s **Claude 3.7 Sonnet**, and Microsoft’s **Majorana 1** ([analyticsvidhya.com](https://www.analyticsvidhya.com/blog/2025/03/generative-ai-launches-of-february/?utm_source=openai)).  
- Chinese AI leader Baidu introduced **Ernie 4.5** and **Ernie X1**—multimodal models claimed to outperform competitors on benchmarks like CCBench and OCRBench; Ernie 4.5 is slated to become open-source by **June 30, 2025** ([globenewswire.com](https://www.globenewswire.com/news-release/2025/04/17/3063915/0/en/Applied-Generative-AI-Course-Launched-by-Interview-Kickstart-2025-Best-GenAI-Course-With-Agentic-AI-Projects-For-Top-AI-Jobs-at-Google-Meta-Netflix-Microsoft-OpenAI-Nvidia.html?utm_source=openai)).

---

###  Enterprise Integration & Ecosystem Uptake

Generative AI has moved beyond speculative hype—enterprises are now embedding it at scale across sectors:

- **Google** enhanced its **Vertex AI Agent Builder** with improved observability and deployment tools, making AI agent management more robust and enterprise-ready ([infoworld.com](https://www.infoworld.com/article/4085736/google-boosts-vertex-ai-agent-builder-with-new-observability-and-deployment-tools.html?utm_source=openai)).
- **AWS** showcased new generative AI and media distribution innovations (via Amazon Bedrock services) at **IBC 2025**, drawing attention to AI’s role in media industries ([aws.amazon.com](https://aws.amazon.com/blogs/media/aws-to-show-new-generative-ai-news-distribution-innovations-at-ibc-2025/?utm_source=openai)).
- **McKinsey’s 2025 Tech Trends** highlighted generative AI as a transformative technology demanding strategic integration across sectors ([mckinsey.com](https://www.mckinsey.com/capabilities/tech-and-ai/our-insights/the-top-trends-in-tech?utm_source=openai)).
- **Governance** has become crucial: frameworks ensuring fairness, transparency, and ethical deployment are integral to sustaining generative AI adoption ([cbia.com](https://www.cbia.com/news/issues-policies/gen-ai-information-governance/?utm_source=openai)).
- **Amazon CEO Andy Jassy** emphasized generative AI’s penetration across the company, reinventing customer experiences and enabling new business possibilities ([aboutamazon.com](https://www.aboutamazon.com/news/company-news/amazon-ceo-andy-jassy-on-generative-ai?utm_source=openai)).

---

###  Emerging Trends & Industry Outlook

- The industry is transitioning to **agentic AI**—autonomous systems capable of reasoning, planning, and acting independently without constant human input. This trend is projected to accelerate into **2026**, with the autonomous AI market expected to reach USD 11.79 billion ([simplilearn.com](https://www.simplilearn.com/top-technology-trends-and-jobs-article?utm_source=openai)).
- According to AI News, generative AI in 2025 is entering a **mature phase**, focusing on dependable, accurate, and efficient systems integrated into everyday workflows ([artificialintelligence-news.com](https://www.artificialintelligence-news.com/news/generative-ai-trends-2025-llms-data-scaling-enterprise-adoption/?utm_source=openai)).

---

###  Research Frontiers & Technical Innovations

- **Chronologically Consistent Generative AI**: A new modeling paradigm ensuring training data is strictly partitioned before a knowledge cutoff to avoid lookahead bias, offering researchers reproducible, bias-free prediction tools ([arxiv.org](https://arxiv.org/abs/2510.11677?utm_source=openai)).
- **EchoGen**: An auto-regressive framework enabling subject-driven visual generation with much lower sampling latency than diffusion models—ideal for high-fidelity yet efficient creative applications ([arxiv.org](https://arxiv.org/abs/2509.26127?utm_source=openai)).
- Domain-specific models like **BF-GAN** illustrate generative AI’s growing application scope beyond content creation—for example, simulating realistic bubbly flow images in engineering contexts ([arxiv.org](https://arxiv.org/abs/2502.06863?utm_source=openai)).

---

###  Summary Table (Key Developments)

| Category               | Notable Developments                                                                 |
|------------------------|--------------------------------------------------------------------------------------|
| **Major Model Releases** | GPT-5, Gemini 2.5 “Nano Banana,” Llama 4 series, Ernie 4.5/X1, open-weight GPT models |
| **Enterprise Adoption**  | Vertex AI upgrades, AWS’s media AI tools, AI governance emphasis, Amazon AI use       |
| **Emerging Trends**      | Agentic AI maturity, dependable models for scalable use                             |
| **Research Advances**    | Bias-reducing models (Chrono-consistent), efficient generation (EchoGen), niche models (BF-GAN) |

---

In summary, as of **November 2025**, generative AI has firmly entered a new era—marked by powerful multimodal models, enterprise readiness, ethical governance, and continued technical innovation. Let me know if you’d like a deep dive into any specific model, application area, or technical innovation.

### Stateful Response

The OpenAI Responses API includes a stateful feature that enables continuity in interactions. 
By using the `response_id`, a conversation can persist across multiple queries, 
allowing users to refine or expand upon previous searches. This is particularly useful for iterative research, 
dynamic content generation, and applications that require follow-up queries based on prior responses.

In [16]:
fetched_response = client.responses.retrieve(response_id=web_search_response.id)
display(Markdown(fetched_response.output_text[:100]))

Here’s an in-depth update on the **latest developments in generative AI** as of **mid-November 2025*

### Continue Query with Web Search

In [17]:
continue_query = 'find different news'

continue_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= continue_query,
    previous_response_id=web_search_response.id,
    tools=[
        {
            "type": "web_search"
        }
    ]
)

In [18]:
display(Markdown(continue_search_response.output_text))

Here is a detailed and structured roundup of **recent developments in generative AI**, as of **November 12, 2025**, highlighting five major news stories. Each section is richly cited and carefully updated.

---

###  TIME Launches “TIME AI Agent” (Nov 10, 2025)
TIME introduced the **TIME AI Agent**, an advanced, unified AI-powered platform built in partnership with Scale AI just two days ago (Nov 10, 2025). This innovation transforms how readers engage with journalism—letting users generate summaries, audio reports, translations, and explore content via conversation and voice. The platform emphasizes editorial integrity through strong moderation and data governance, positioning it as the next phase in interactive news consumption.([time.com](https://time.com/7332572/the-story-behind-the-time-ai-agent/?utm_source=openai))

---

###  Generative AI in Gaming: Steam Sees Massive Surge (July 2025)
A study revealed a 681% increase year-over-year in generative AI use among Steam games. As of mid-July 2025, **7,818 titles**—roughly **7% of Steam’s 114,000-game catalog**—disclose using GenAI in development, marking **one in five new releases in 2025** as generative AI-enabled. Visual assets remain the most common use (60%), followed by audio, narrative, marketing, and code. Live AI during gameplay is rising, though developers remain cautious due to consumer trust concerns.([tomshardware.com](https://www.tomshardware.com/video-games/pc-gaming/1-in-5-steam-games-released-in-2025-use-generative-ai-up-nearly-700-percent-year-on-year-7-818-titles-disclose-genai-asset-usage-7-percent-of-entire-steam-library?utm_source=openai))

---

###  Tencent Open-Sources 3D AI Tools (March 2025)
In March, **Tencent** released five open-source AI models for generating 3D visuals, based on its **Hunyuan3D-2.0** architecture. These “turbo” models can create high-quality 3D content in just 30 seconds. The move positions Tencent as a strong competitor to U.S. firms by democratizing access to powerful AI tools for designers and game developers.([reuters.com](https://www.reuters.com/technology/artificial-intelligence/tencent-expands-ai-push-with-open-source-3d-generation-tools-2025-03-18/?utm_source=openai))

---

###  NY Times Licensing Deal with Amazon (May 2025)
In May, **The New York Times** struck its first generative AI licensing deal—partnering with **Amazon**. This multi-year agreement allows Amazon to incorporate summaries and selected content from NYT, The Athletic, and NYT Cooking into its products (e.g., Alexa) and use these materials to train its AI models. The deal follows litigation against Microsoft and OpenAI, reinforcing the Times' commitment to monetizing and protecting its journalism.([theverge.com](https://www.theverge.com/news/676291/new-york-times-ai-deal-amazon?utm_source=openai))

---

###  Expanding Impact: Generative AI in Search and Market Trends (2025)
- **Generative Engine Optimization (GEO) Market Growth**: The GEO market is estimated at **USD 848 million in 2025**, projected to grow at a **50.5% CAGR** to **USD 33.68 billion by 2034**.([en.wikipedia.org](https://en.wikipedia.org/wiki/Generative_engine_optimization?utm_source=openai))  
- **AI Overviews in Google Search**: As of August 2025, “AI Overviews” now appear in **over 50% of Google search results**, up from just 25% ten months prior.([en.wikipedia.org](https://en.wikipedia.org/wiki/Generative_engine_optimization?utm_source=openai))  
- **Consumer Adoption**: AI tool usage among consumers grew from 14% to **29.2% (early 2024 to August 2025)**. As of March 2025, ChatGPT held a **62.5% share** of consumer AI tools with **525.9 million unique visitors**, and approximately **58% of U.S. adults** experienced AI-generated search summaries.([en.wikipedia.org](https://en.wikipedia.org/wiki/Generative_engine_optimization?utm_source=openai))

---

### Summary Table

| Topic                         | Key Highlights                                                                 |
|-------------------------------|--------------------------------------------------------------------------------|
| **TIME AI Agent**             | Launched Nov 10, 2025; interactive news platform with AI summarization and voice features. |
| **Generative AI in Gaming**   | 2025 marked a 681% increase; 1 in 5 new Steam games use GenAI.                 |
| **Tencent's 3D Models**       | Released open-source Hunyuan3D-2.0 models for fast 3D asset generation.        |
| **NY Times–Amazon Deal**      | May 2025 agreement to license content and train AI models.                    |
| **Search & Market Trends**    | GEO market booming; AI Overviews in 50%+ of Google searches; rising AI tool adoption. |

---

These developments collectively illustrate the accelerating integration of generative AI across media, gaming, enterprise, and everyday tech usage. Let me know if you'd like to explore any of these stories further or dive into specific technical, business, or ethical implications.

### Combining File Search and Web Search

This is an example of using file search to analyze private data and web search to retrieve public or the latest data. 
The Responses API allows developers to integrate these tools to enhance retrieval-augmented generation (RAG) applications. 
By combining file search with web search, users can leverage structured internal knowledge while also retrieving real-time 
information from external sources, ensuring comprehensive and up-to-date responses. 

In [19]:
combined_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= query,
    temperature = 0,
    instructions="Retrieve the results from the file search first, and use the web search tool to expand the results with news resources",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
    },
        {
            "type": "web_search"
        }
    ]
)

In [20]:
display(Markdown(combined_search_response.output_text))

Recent developments in generative AI include several key advancements and applications:

1. **AI in Creative Industries**: Generative AI is being used to create cinematic videos and food commercials, significantly reducing production time and costs.

2. **Business Innovation**: Companies are leveraging generative AI to innovate in areas like supply chain disruption analysis and enterprise architecture consulting.

3. **Healthcare Transformation**: Generative AI is supporting doctors with data interpretation and preliminary analysis, enhancing patient care.

4. **Market Growth**: The global generative AI market is expected to reach $1.18 billion this year, highlighting its growing importance across various sectors.

5. **Ethical and Regulatory Considerations**: There is ongoing discussion about the regulation and ethics of generative AI, particularly concerning its impact on intellectual property and content ownership.

These developments indicate that generative AI is not only advancing technologically but also becoming increasingly integrated into diverse industries, driving innovation and raising important ethical questions.

# 🧩 Try It Yourself: Two-Step RAG (Private Data + Combined Search)

## Step 1 — Upload & Create Vector Store
1. Upload a short text file (e.g., `my_notes.txt`) to your notebook instance.  
2. Create a **vector store** and **ingest** your uploaded file.  
3. Run a simple test query to verify retrieval:  

In [22]:
with open('raab test overview cards and more .txt', 'rb') as f:
    file = client.files.create(
        file=f,            # file-like object
        purpose="assistants"
    )

file_id = file.id
print(file_id)

file-WNCPDNkn3CmvhzETDrzQnj


In [23]:
attach_status =client.vector_stores.files.create(
    vector_store_id=vector_store_id,
    file_id=file_id
            )

print(attach_status.id)

file-WNCPDNkn3CmvhzETDrzQnj


In [24]:
query='facts on bacteria'

In [25]:
search_results = client.vector_stores.search(
    vector_store_id=vector_store_id,
    query=query
)

for result in search_results.data[:5]:
    print(result.content[0].text[:100] + '\n Relevant score: ' + str(result.score))

Card 1: Bacteria Overview
* Definition: Single-celled microorganisms; can survive without a host in
 Relevant score: 0.6826254397674981
________________








Card 1: Overview of Biological Agents
* Categories:
   * Bacteri
 Relevant score: 0.6377566937476328
* Gastrointestinal: Abdominal pain, severe vomiting.
________________


Card 7: Plague Overview
 Relevant score: 0.6144790266650119
4. Medical and Diagnostic Features:
   * Anthrax: Cutaneous, gastrointestinal, inhalational forms; 
 Relevant score: 0.5895079062505587
* Smallpox: Human-to-human only, higher fatality.
* Key Difference: Monkeypox spread from animals; 
 Relevant score: 0.45267519872519096


## Step 2 — Combine File Search with Web Search
1. Enable both **file_search** and **web_search** in the Responses API.  
2. Use a prompt that asks the model to merge insights from both sources.  
   > Example: “Using my uploaded notes and the latest web information, summarize the current trends on this topic.”  
3. Review how the answer from your file and **current info** from the web.

✅ You’ve created a RAG system that combines **private** and **public** data for comprehensive, up-to-date analysis.


In [27]:

file_search_response = client.responses.create(
    input= query,
    model="gpt-4o",
    temperature = 0,
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
    }]
)

In [28]:
display(Markdown(file_search_response.output_text))

Here are some facts about bacteria:

1. **Definition**: Bacteria are single-celled microorganisms that can survive independently in oxygen-rich environments.

2. **Control**: They are typically controlled by antibiotics.

3. **Examples**: Notable examples of bacteria include those causing the plague and anthrax.

4. **Biological Warfare**: Bacteria have been used in biological warfare due to their ability to cause disease.

5. **Medical and Diagnostic Features**: Bacteria like anthrax and plague have specific forms and symptoms, and they are treatable with antibiotics.

In [29]:
web_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= query,
    tools=[
        {
            "type": "web_search"
        }
    ]
)

In [30]:
display(Markdown(web_search_response.output_text))

Here are some interesting facts about bacteria:

1. **Diversity**: Bacteria are incredibly diverse, with millions of species that vary in shape, size, and habitat.

2. **Ubiquity**: They are found everywhere on Earth, from the deepest oceans to the arid deserts and even inside our bodies.

3. **Size**: Typically, bacteria are about 0.5 to 5.0 micrometers in size, which is much smaller than most eukaryotic cells.

4. **Shapes**: Common shapes of bacteria include rods (bacilli), spheres (cocci), spirals (spirilla), and commas (vibrios).

5. **Reproduction**: Bacteria primarily reproduce asexually through a process called binary fission.

6. **Genetic Exchange**: They can exchange genetic material through processes like conjugation, transformation, and transduction, contributing to genetic diversity.

7. **Beneficial Roles**: Many bacteria are beneficial; for example, they play a crucial role in digestion, nutrient cycling, and biodegradation.

8. **Pathogenicity**: Some bacteria can cause diseases in humans, animals, and plants, such as tuberculosis, strep throat, and bacterial meningitis.

9. **Antibiotics**: Bacterial infections are often treated with antibiotics, but overuse has led to antibiotic resistance, a significant public health concern.

10. **Extremophiles**: Certain bacteria, known as extremophiles, can thrive in extreme conditions, such as hot springs, acidic environments, and radioactive waste.

11. **Biofilms**: Bacteria can form biofilms, complex communities attached to surfaces, which can be more resistant to antibiotics.

12. **Role in Food**: Bacteria are used in food production, such as in the fermentation process to produce yogurt, cheese, and vinegar.

Understanding bacteria is crucial for fields such as medicine, agriculture, and biotechnology.

In [31]:
combined_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= query,
    temperature = 0,
    instructions="Retrieve the results from the file search first, and use the web search tool to expand the results with news resources",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
    },
        {
            "type": "web_search"
        }
    ]
)

In [32]:
display(Markdown(combined_search_response.output_text))

Here are some facts about bacteria:

1. **Definition**: Bacteria are single-celled microorganisms that can survive independently in oxygen-rich environments.

2. **Control**: They are typically controlled by antibiotics.

3. **Examples**: Some well-known bacteria include those causing the plague and anthrax.

4. **Biological Warfare**: Bacteria have been used in biological warfare due to their ability to cause disease and death.

5. **Medical and Diagnostic Features**: Bacteria like anthrax and plague have specific forms and symptoms, and they are treatable with antibiotics.

If you need more detailed or specific information, feel free to ask!